# CNN 😎

#### Libraries

In [1]:
import utilities as ff
import matplotlib.pyplot as plt
import numpy as np
import pathlib

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow import config as tfconfig

from keras.layers import *
from keras.models import *
from keras.losses import *
from keras.optimizers import *
from keras.utils import *


In [3]:
tfconfig.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices()[1])

name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1386046875
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10731660993588102717
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1050, pci bus id: 0000:02:00.0, compute capability: 6.1"
xla_global_id: 416903419



#### HyperParameter

In [ ]:
no_classes = 2
seed = 42

In [ ]:
hyper_point_batch_size = [8, 32, 128, 512, ]
hyper_point_image_squared_size = [32, 64, 128, 256, ]
hyper_point_color_mode = ["grayscale", "rgb"]


In [5]:
batch_size = hyper_point_batch_size[1]
image_squared_size = hyper_point_image_squared_size[2]
color_mode = hyper_point_color_mode[0]

#### Dataset

In [6]:
train_rgb, test_rgb = ff.ready_to_be_used_dataset(
    batch_size=batch_size,
    image_squared_size=image_squared_size,
    color_mode="rgb",
)


Found 24770 files belonging to 2 classes.
Using 19816 files for training.
Found 24770 files belonging to 2 classes.
Using 4954 files for validation.


In [7]:
train_bw, test_bw = ff.ready_to_be_used_dataset(
    batch_size=batch_size,
    image_squared_size=image_squared_size,
    color_mode="grayscale",
)


Found 24770 files belonging to 2 classes.
Using 19816 files for training.
Found 24770 files belonging to 2 classes.
Using 4954 files for validation.


In [8]:
# image_batch, labels_batch = next(iter(train))
# print(image_batch.shape, labels_batch.shape)


#### RGB Model definition

https://www.tensorflow.org/tutorials/images/cnn

In [9]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation="relu", input_shape=(image_squared_size, image_squared_size, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(Flatten())
model.add(Dense(64, activation="relu"))
model.add(Dense(16, activation="relu"))
model.add(Dense(2))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 flatten (Flatten)           (None, 238144)            0         
                                                                 
 dense (Dense)               (None, 64)                15241280  
                                                                 
 dense_1 (Dense)             (None, 2)                 130       
                                                        

In [10]:
model.compile(
    optimizer="adam",
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)


#### Training

In [11]:
history = model.fit(train_rgb, epochs=10, validation_data=test_rgb)

Epoch 1/10
472/472 [==============================] - 52s 100ms/step - loss: 0.7058 - accuracy: 0.6407 - val_loss: 0.5876 - val_accuracy: 0.6875
Epoch 2/10
472/472 [==============================] - 46s 96ms/step - loss: 0.5200 - accuracy: 0.7408 - val_loss: 0.5962 - val_accuracy: 0.7033
Epoch 3/10
472/472 [==============================] - 46s 97ms/step - loss: 0.3477 - accuracy: 0.8445 - val_loss: 0.6839 - val_accuracy: 0.7148
Epoch 4/10
472/472 [==============================] - 46s 96ms/step - loss: 0.1584 - accuracy: 0.9386 - val_loss: 1.0129 - val_accuracy: 0.7182
Epoch 5/10
287/472 [=================>............] - ETA: 16s - loss: 0.0956 - accuracy: 0.9657

KeyboardInterrupt: 

#### Performance evaluation

In [ ]:
def performance_plot(history):
    plt.figure(figsize=(16, 6))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history["loss"])
    plt.plot(history.history["val_loss"])
    plt.ylabel("loss", size=12)
    plt.xlabel("epoch", size=12)
    plt.legend(["train", "val"], fontsize=10)

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history["accuracy"])
    plt.plot(history.history["val_accuracy"])
    plt.ylabel("accuracy", size=12)
    plt.xlabel("epoch", size=12)
    plt.legend(["train", "val"], fontsize=10)

    plt.show()


performance_plot(history)
